In [1]:
import pickle
import torch as t
from torch.utils import data

In [3]:
with open('/home/LAB/wangd/graduation_project/ranked list truncation/dataset/robust04_gt.pkl', 'rb') as f: 
    gt = pickle.load(f)

In [4]:
DATASET_BASE = '/home/LAB/wangd/graduation_project/ranked list truncation/dataset'
def data_prepare(dataset_name: str):
    with open('{}/{}_train.pkl'.format(DATASET_BASE, dataset_name), 'rb') as f:
        train_data_raw = pickle.load(f)
    with open('{}/{}_test.pkl'.format(DATASET_BASE, dataset_name), 'rb') as f:
        test_data_raw = pickle.load(f)
    with open('{}/robust04_gt.pkl'.format(DATASET_BASE), 'rb') as f:
        gt = pickle.load(f)
        for key in gt: gt[key] = set(gt[key])

    X_train, X_test, y_train, y_test = [], [], [], []
    for key in train_data_raw:
        scores = list(train_data_raw[key].values())
        is_rel = list(map(lambda x: 1 if x in gt[key] else 0, train_data_raw[key].keys()))
        X_train.append(scores)
        y_train.append(is_rel)
    for key in test_data_raw:
        scores = list(test_data_raw[key].values())
        is_rel = list(map(lambda x: 1 if x in gt[key] else 0, test_data_raw[key].keys()))
        X_test.append(scores)
        y_test.append(is_rel)

    X_train, X_test = t.unsqueeze(t.Tensor(X_train), dim=1).permute(0, 2, 1), t.unsqueeze(t.Tensor(X_test), dim=1).permute(0, 2, 1)
    y_train, y_test = t.Tensor(y_train), t.Tensor(y_test)
    
    d00, d01, _ = X_train.shape
    d10, d11, _ = X_test.shape
    position_embedding_train, position_embedding_test = t.randn(d00, d01, 127), t.randn(d10, d11, 127)
    X_train = t.cat((X_train, position_embedding_train), dim=2)
    X_test = t.cat((X_test, position_embedding_test), dim=2)

    return X_train, X_test, y_train, y_test

In [5]:
xtr, xte, ytr, yte = data_prepare('drmm_tks_s1')

In [6]:
xtr.shape

torch.Size([194, 300, 128])

In [11]:
xtr[0][0][:5]


tensor([ 8.8141, -0.3287, -0.1865, -1.9727,  1.0374])

In [12]:
xtr[0][1][:5]

tensor([ 8.4056, -0.5739,  0.3004, -0.8791,  0.6017])

In [13]:
xtr[0][2][:5]

tensor([ 7.6534,  1.2669, -0.1602, -0.2902, -0.2273])

In [14]:
dataset_name = 'drmm_tks_s1'
with open('{}/{}_train.pkl'.format(DATASET_BASE, dataset_name), 'rb') as f:
    train_data_raw = pickle.load(f)
with open('{}/{}_test.pkl'.format(DATASET_BASE, dataset_name), 'rb') as f:
    test_data_raw = pickle.load(f)

In [22]:
for item in list(train_data_raw['337'].keys())[:5]:
    print(item in gt['337'])

True
True
True
True
True


In [18]:
list(train_data_raw['337'].values())[:5]

[8.814115524291992,
 8.405571937561035,
 7.653446197509766,
 7.354440689086914,
 7.292997360229492]